In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#prova

In [ ]:
import utils.data as data

dataset = data.Marconi100Dataset()

100%|██████████| 250/250 [00:36<00:00,  6.86it/s]


In [ ]:
dataset[0]

,timestamp,avg:ambient,var:ambient,max:ambient,min:ambient,avg:dimm0_temp,var:dimm0_temp,max:dimm0_temp,min:dimm0_temp,avg:dimm10_temp,...,avg:swap_free,var:swap_free,max:swap_free,min:swap_free,avg:swap_total,var:swap_total,max:swap_total,min:swap_total,label,New_label
index,,,,,,,,,,,,,,,,,,,,,
24,2020-06-01 00:00:00,25.575000,0.005000,25.6,25.4,32.250000,0.214286,33,32,34.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
27,2020-06-01 00:15:00,25.560000,0.006857,25.6,25.4,32.333333,0.238095,33,32,34.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
30,2020-06-01 00:30:00,25.600000,0.000000,25.6,25.6,32.533333,0.266667,33,32,34.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
33,2020-06-01 00:45:00,25.600000,0.000000,25.6,25.6,32.071429,0.071429,33,32,34.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
36,2020-06-01 01:00:00,25.571429,0.005275,25.6,25.4,32.071429,0.071429,33,32,34.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96177,2021-04-30 20:45:00,24.533333,0.009524,24.6,24.4,32.000000,0.000000,32,32,33.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
96180,2021-04-30 21:00:00,24.600000,0.000000,24.6,24.6,32.000000,0.000000,32,32,33.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0
96183,2021-04-30 21:15:00,24.600000,0.000000,24.6,24.6,32.000000,0.000000,32,32,33.0,...,4194240.0,0.0,4194240,4194240,4194240,0.0,4194240,4194240,0,0


In [ ]:
dataset[0].describe()

,avg:ambient,var:ambient,max:ambient,min:ambient,avg:dimm0_temp,var:dimm0_temp,max:dimm0_temp,min:dimm0_temp,avg:dimm10_temp,var:dimm10_temp,...,avg:swap_free,var:swap_free,max:swap_free,min:swap_free,avg:swap_total,var:swap_total,max:swap_total,min:swap_total,label,New_label
count,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,13243.000000,...,1.324300e+04,1.324300e+04,1.324300e+04,1.324300e+04,13243.0,13243.0,13243.0,13243.0,13243.000000,13243.000000
mean,23.532033,0.008234,23.610904,23.450683,30.702113,0.020381,30.760175,30.643510,32.003146,0.016253,...,3.863859e+06,1.478030e+07,3.863923e+06,3.863817e+06,4194240.0,0.0,4194240.0,4194240.0,0.008080,0.040172
std,1.272196,0.050238,1.262907,1.291913,1.334301,0.065988,1.327003,1.356598,1.059163,0.059249,...,2.600040e+05,1.350319e+09,2.599753e+05,2.600922e+05,0.0,0.0,0.0,0.0,0.089527,0.280600
min,16.440000,0.000000,16.600000,16.400000,23.000000,0.000000,23.000000,23.000000,27.000000,0.000000,...,2.957248e+06,0.000000e+00,2.957248e+06,2.957248e+06,4194240.0,0.0,4194240.0,4194240.0,0.000000,0.000000
25%,23.026667,0.000000,23.200000,23.000000,30.000000,0.000000,30.000000,30.000000,32.000000,0.000000,...,3.719232e+06,0.000000e+00,3.719232e+06,3.719232e+06,4194240.0,0.0,4194240.0,4194240.0,0.000000,0.000000
50%,23.666667,0.004952,23.800000,23.600000,31.000000,0.000000,31.000000,31.000000,32.000000,0.000000,...,3.741056e+06,0.000000e+00,3.741056e+06,3.741056e+06,4194240.0,0.0,4194240.0,4194240.0,0.000000,0.000000
75%,24.493333,0.009524,24.600000,24.400000,32.000000,0.000000,32.000000,32.000000,33.000000,0.000000,...,4.194240e+06,0.000000e+00,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0,0.000000,0.000000
max,30.040000,3.867810,30.800000,29.200000,34.466667,2.142857,35.000000,34.000000,38.466667,1.980952,...,4.194240e+06,1.484832e+11,4.194240e+06,4.194240e+06,4194240.0,0.0,4194240.0,4194240.0,1.000000,2.000000
